<a href="https://colab.research.google.com/github/JoseJ09/Data-Ingestion-to-AWS-S3/blob/main/Data_Ingestion_to_AWS_S3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install boto3
!pip install pickle
!pip install yaml
!pip install sodapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import pickle
import boto3

from datetime import date
from sodapy import Socrata

In [ ]:
with open('credentials.yaml', 'r') as f:
    config = yaml.safe_load(f)

token = config['api_chicago']['app_token']
username = config['api_chicago']['username']
password = config['api_chicago']['password']
matricula = config['iexe']['matricula']


In [ ]:
# code for dataset of food inspections
chicago_dataset = "4ijn-s7e5"

### 1. Función get_client

In [ ]:
def get_client():
    client = Socrata("data.cityofchicago.org",
                     token,
                     username=username,
                     password=password)

    return client

*italicized text*### 2. Ingesta inicial

Bajar los datos de la API de inspecciones de establecimientos de comida.

In [ ]:
def ingesta_inicial(chicago_dataset, client, limit):
    datasets = client.get(chicago_dataset, limit=limit, offset=0, order='inspection_date')

    return datasets

In [ ]:
client = get_client()
datasets = ingesta_inicial(chicago_dataset, client, 300000)

In [ ]:
len(datasets)

252043

In [ ]:
datasets[0]

{'inspection_id': '104236',
 'dba_name': 'TEMPO CAFE',
 'aka_name': 'TEMPO CAFE',
 'license_': '80916',
 'facility_type': 'Restaurant',
 'risk': 'Risk 1 (High)',
 'address': '6 E CHESTNUT ST ',
 'city': 'CHICAGO',
 'state': 'IL',
 'zip': '60611',
 'inspection_date': '2010-01-04T00:00:00.000',
 'inspection_type': 'Canvass',
 'results': 'Fail',
 'violations': '18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: All necessary control measures shall be used to effectively minimize or eliminate the presence of rodents, roaches, and other vermin and insects on the premises of all food establishments, in food-transporting vehicles, and in vending machines.    Front patio door not rodent proofed. Basement rear door not rodent proofed. Must seal and maintain all outer openings. | 19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE AREA; CLEAN, RODENT PROOF, ALL CONTAINERS COVERED - Comments: The area out

### 3. Almacenamiento de ingesta inicial

In [ ]:
def guardar_ingesta(bucket, bucket_path, dataset):
    session = boto3.Session(
        aws_access_key_id = config['s3']['aws_access_key_id'],
        aws_secret_access_key = config['s3']['aws_secret_access_key'],
        aws_session_token= config['s3']['aws_session_token']
    )

    s3 = session.resource('s3')
    s3.Object(bucket, bucket_path).put(Body=dataset)

### 3a. Creación de Bucket
##### creamos la variable con la fecha de hoy

In [ ]:
TODAY = date.today()
if not 'bucket_creado' in locals():
  print("creando")
  session = boto3.Session(
    aws_access_key_id = config['s3']['aws_access_key_id'],
    aws_secret_access_key = config['s3']['aws_secret_access_key'],
    aws_session_token= config['s3']['aws_session_token']
  )
  s3 = session.resource('s3')
  nombre_bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
  s3.create_bucket(Bucket=nombre_bucket)
  bucket_creado = True

In [ ]:
 nombre_bucket

'aplicaciones-cd-1-mcda22c007'

In [ ]:
pickle_data = pickle.dumps(datasets)

In [ ]:
bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
key = "ingesta/inicial/inspecciones-historicas-" + str(TODAY) + ".pkl"

guardar_ingesta(bucket, key, pickle_data)

Guardar el dataset en un pickle en un bucket de s3, el nombre del archivo dete terminar con el día en el que se realizó la ingesta (obtenido de manera dinámica).

### 4. Ingestas consecutivas

La siguiente vez que se ingestan los datos se tendrá que pedir específicamente a partir de la fecha desde la que no tienes datos.

In [ ]:
def ingesta_consecutiva(chicago_dataset, client, fecha, limit):
    new_dataset = client.get(chicago_dataset, limit=limit, where="inspection_date>='{}'".format(fecha))

    return new_dataset

In [ ]:
client = get_client()

In [ ]:
new_dataset = ingesta_consecutiva(chicago_dataset, client, '2020-11-03', 1000)

In [ ]:
new_dataset[0]

{'inspection_id': '2456231',
 'dba_name': 'THAI ROOM RESTAURANT INC.',
 'aka_name': 'THAI ROOM RESTAURANT',
 'license_': '9571',
 'facility_type': 'Restaurant',
 'risk': 'Risk 1 (High)',
 'address': '4022 N WESTERN AVE ',
 'city': 'CHICAGO',
 'state': 'IL',
 'zip': '60618',
 'inspection_date': '2020-11-03T00:00:00.000',
 'inspection_type': 'Canvass',
 'results': 'Pass w/ Conditions',
 'violations': '5. PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - Comments: OBSERVED SUPPLIES MISSING FROM VOMIT AND DIARRHEA CLEAN UP KIT AND PROCEDURE. INSTRUCTED TO PROVIDE ALL NECESSARY SUPPLIES AS OUTLINED IN POLICY. PRIORITY FOUNDATION. 7-38-005. NO CITATION ISSUED. | 38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - Comments: OBSERVED THREE DEAD COCKROACHES ON GLUEBOARD LOCATED UNDERNEATH 3-COMPARTMENT SINK. MUST REMOVE. | 53. TOILET FACILITIES: PROPERLY CONSTRUCTED, SUPPLIED, & CLEANED - Comments: OBSERVED NO LID ON TRASH CAN IN WOMEN?S BATHROOM. MUST PROVIDE.',
 'latitude': '41.954759040

In [ ]:
len(new_dataset)

1000

### 5. Almacenamiento de ingestas consecutivas

In [ ]:
pickled_new_data = pickle.dumps(new_dataset)

In [ ]:
bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
key = "ingesta/consecutiva/inspecciones-consecutivas-" + str(TODAY) + ".pkl"

guardar_ingesta(bucket, key, pickled_new_data)